# Create observing files for session starting on 2024-11-23 UTC

In [ ]:
from functools import partial
import importlib
from sqlite3 import connect
import sys

import astroplan as ap
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
from icecream import ic
import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


ModuleNotFoundError: No module named 'astropaul'

In [15]:
importlib.reload(tlc)
importlib.reload(lbt)

conn = connect("../../django/TargetDB/db.sqlite3")
html_dir = "html"

session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2024-11-23", "2024-11-27 12:00")

already_viewed_2024B = {
    "TIC 89278612",  # 2024-09-20 05:17:18
    "TIC 470710327",  # 2024-09-20 06:46:13
    "TIC 367448265",  # 2024-09-20 08:35:06
    "TIC 260056937",  # 2024-09-20 10:39:05
    "TIC 392229331",  # 2024-09-20 10:53:46
    "TIC 286470992",  # 2024-09-20 11:17:00
    "TIC 470710327",  # 2024-09-22 04:07:37
    "TIC 278352276",  # 2024-10-30 01:34:24
    "TIC 414026507",  # 2024-10-31 02:12:56
    "TIC 427092089",  # 2024-10-31 02:22:29
    "TIC 52856877",  # 2024-11-01 06:02:56
    # "TIC 283940788", # 2024-11-23
    # "TIC 307119043", # 2024-11-23
    # "TIC 454140642", # 2024-11-23
    # "TIC 336882813", # 2024-11-23
    # "TIC 123098844", # 2024-11-24 (UT 01:10)
    # "TIC 389836747", # 2024-11-24 (UT 06:30)
    # "TIC 336882813", # 2024-11-24 (UT 08:35)
}
already_viewed_2024B.remove("TIC 470710327") # this one is special and deserves additional followup observations

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": False,
    "priority": "(see grid)",
}

name = "2024-11-23 LBT Observing List"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"] == "QuadEB")),
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"].isin(already_viewed_2024B) == False)),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    tlc.add_speckle,
    tlc.add_spectra,
    tlc.add_lists,
    tlc.add_ephemerides,
    partial(tlc.filter_targets, criteria=lambda df: (df["Spectra Count"] > 0)),
    partial(tlc.filter_targets, criteria=lambda df: (df["List LBT 2024B"])),
    partial(tlc.concat_dataframe, other_df=pd.read_csv("Calibration Targets.csv")),
    partial(tlc.add_observability, observing_session=session, calc_max_altitude=True, calc_moon_distance=True),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Every Night"])),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Max Alt"] > 40)),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: np.isnan(df["Teff"])),
    partial(lbt.add_pepsi_params, **pepsi_args),
    partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 660),
]
tl = creator.calculate(verbose=False)
tl.target_list["PEPSI notes"] = [type.replace("QuadEB", "Science Target") for type in tl.target_list["Target Type"]]
tl.to_html(f"{html_dir}/{name}.html")
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list)

print(tl.summarize())
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
tl.target_list

2024-11-23 LBT Observing List
22 targets:
      10 QuadEB
       9 RV Standard
       3 Telluric Standard
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Speckle Count: (1, 0)
    Spectra Count: (1, 0)
    List : (0, 10)
    Observable: (4, 15)
    PEPSI : (3, 6)
Other tables sizes:
     783 Speckle (7 cols)
     103 Spectra (8 cols)
     625 Ephem (7 cols)
       5 Lunar Phases (2 cols)

74.0 minutes


,Target Name,Target Source,Target Type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,...,Observable Min Moon Dist,PEPSI fiber,PEPSI cd_blue,PEPSI cd_blue_num_exp,PEPSI cd_red,PEPSI cd_red_num_exp,PEPSI snr,PEPSI exp_time,PEPSI priority,PEPSI notes
4219,TIC 25818450,Kostov 2022 arXiv:2202.05790,QuadEB,352.743444,53.069150,3.57360,-1.172130,1.065210,11.782,7172.00,...,111.374395,300,3,1,6,1,100,627.8,(see grid),Science Target
4225,TIC 470710327,Kostov 2022 arXiv:2202.05790,QuadEB,357.329052,61.962787,0.60000,-2.500000,NaN,9.650,8986.00,...,102.216071,300,3,1,6,1,100,79.9,(see grid),Science Target
4232,TIC 123098844,Kostov 2022 arXiv:2202.05790,QuadEB,279.572843,44.698600,3.48250,-9.104270,1.309250,11.136,6761.00,...,91.923099,300,3,1,6,1,100,348.6,(see grid),Science Target
4236,TIC 322727163,Kostov 2022 arXiv:2202.05790,QuadEB,309.716625,50.466821,2.80000,-7.500000,NaN,10.997,7876.56,...,111.954272,300,3,1,6,1,100,282.4,(see grid),Science Target
4277,TIC 454140642,Kostov 2022 arXiv:2202.05790,QuadEB,64.773473,0.900042,2.82736,-10.846600,2.712150,10.409,6592.00,...,88.181604,300,3,1,6,1,100,178.4,(see grid),Science Target
4280,TIC 336882813,Kostov 2022 arXiv:2202.05790,QuadEB,92.390615,14.628986,-1.00000,-10.200000,NaN,11.788,6121.22,...,58.744576,300,3,1,6,1,100,654.1,(see grid),Science Target
4300,TIC 307119043,Kostov 2022 arXiv:2202.05790,QuadEB,14.827524,51.221643,7.56019,-4.108770,2.198800,9.940,7815.00,...,105.866071,300,3,1,6,1,100,111.4,(see grid),Science Target
4306,TIC 283940788,Kostov 2022 arXiv:2202.05790,QuadEB,8.851436,62.901596,-2.72664,-0.153866,0.076539,11.774,7384.00,...,98.773130,300,3,1,6,1,100,603.5,(see grid),Science Target
4313,TIC 328181241,Kostov 2022 arXiv:2202.05790,QuadEB,43.153958,3.347882,-4.66787,-9.873120,1.974080,11.636,5183.00,...,108.538295,300,3,1,6,1,100,617.8,(see grid),Science Target
4322,TIC 389836747,Kostov 2022 arXiv:2202.05790,QuadEB,23.295470,61.585307,-6.60000,-2.400000,NaN,10.718,8874.00,...,95.265896,300,3,1,6,1,100,213.8,(see grid),Science Target


In [14]:
importlib.reload(tlc)
importlib.reload(pr)
importlib.reload(ph)

illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.75),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.85),
    ((45, 60), 0.95),
    ((60, 90), 1),
]

phase_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|R": 0.9,
    "R|T": 1,
    "T|T": 0.1,
}

overall_weightings = {"Moon": 1, "Altitude": 1, "Phase": 1, "HQND": 1}

session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2024-11-23", "2024-11-27 12:00")
science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=1 * u.hour)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl, weightings=overall_weightings)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pl.categorize_priorities(
    bins=[0.00, 0.10, 0.25, 0.5, 1.00], labels=["", "*", "* *", '<p style="background-color: #ADD8E6;">* * *</p>']
)
pl.to_html(dir=html_dir)
print(np.max(pl.numerical_priorities[0]))
pl.numerical_priorities[0]

 '2024-11-23T03:00:00.000000000' '2024-11-23T04:00:00.000000000'
 '2024-11-23T05:00:00.000000000' '2024-11-23T06:00:00.000000000'
 '2024-11-23T07:00:00.000000000' '2024-11-23T08:00:00.000000000'
 '2024-11-23T09:00:00.000000000' '2024-11-23T10:00:00.000000000'
 '2024-11-23T11:00:00.000000000' '2024-11-23T12:00:00.000000000'
 '2024-11-23T13:00:00.000000000'], obsgeoloc=[( 4531761.93281974, -2902830.41825555, 3416902.09422008),
 ( 5130134.88202984, -1629476.86071101, 3415410.9309097 ),
 ( 5377558.91714113,  -244463.28627994, 3414763.29614902),
 ( 5257080.5789324 ,  1157309.36434554, 3415003.57191603),
 ( 4776955.02876455,  2479791.82081197, 3416115.30147688),
 ( 3970080.40604897,  3632367.76605365, 3418022.3164909 ),
 ( 2891743.65221052,  4536062.85401997, 3420593.95553897),
 ( 1615832.25697738,  5128956.0211352 , 3423654.01653873),
 (  229771.49763205,  5370422.30915874, 3426992.8296621 ),
 (-1171465.92910496,  5243916.48180566, 3430381.62354898),
 (-2491867.42334283,  4758106.70209062, 

0.85


,TIC 123098844,TIC 322727163,TIC 25818450,TIC 470710327,TIC 283940788,TIC 307119043,TIC 389836747,TIC 328181241,TIC 454140642,TIC 336882813
2024-11-23 01:00:00,0.76,0.60000,0.375,0.28500,0.57000,0.35625,0.25500,0.00000,0.00,0.00000
2024-11-23 02:00:00,0.68,0.60000,0.300,0.28500,0.64125,0.57000,0.35625,0.00000,0.00,0.00000
2024-11-23 03:00:00,0.00,0.35625,0.300,0.30000,0.64125,0.60000,0.35625,0.31875,0.00,0.00000
2024-11-23 04:00:00,0.00,0.25500,0.300,0.35625,0.64125,0.60000,0.37500,0.35625,0.00,0.00000
2024-11-23 05:00:00,0.00,0.00000,0.285,0.35625,0.57000,0.60000,0.37500,0.35625,0.68,0.00000
2024-11-23 06:00:00,0.00,0.00000,0.285,0.35625,0.57000,0.60000,0.35625,0.37500,0.76,0.51000
2024-11-23 07:00:00,0.00,0.00000,0.255,0.51000,0.57000,0.57000,0.35625,0.35625,0.76,0.57000
2024-11-23 08:00:00,0.00,0.00000,0.000,0.51000,0.63750,0.51000,0.35625,0.35625,0.76,0.60000
2024-11-23 09:00:00,0.00,0.00000,0.000,0.00000,0.00000,0.51000,0.25500,0.31875,0.76,0.60000
2024-11-23 10:00:00,0.00,0.00000,0.000,0.00000,0.00000,0.00000,0.00000,0.00000,0.85,0.37500


In [280]:
from itables import JavascriptFunction, show

show(
    pl.categorized_priorities[1],
    columnDefs=[
        {
            "targets": "_all",
            "createdCell": JavascriptFunction(
                """
function (td, cellData, rowData, row, col) {
    if (cellData == "* *") {
        $(td).css('background-color', '#ADD8E6')
    }
}
"""
            ),
        }
    ],
)

In [250]:
pl.numerical_priorities[0].style \
    .map(lambda val: "background-color: light-gray" if val > 0.001 else "background-color: black") \
    .format("{:.3f}") \
    .format_index("{:%H:%M}") \
    # .background_gradient(axis=None, cmap="YlOrRd")

pl.categorized_priorities[1]

TIC 123098844 TIC 322727163 TIC 25818450 TIC 470710327  \
2024-11-24 01:00:00  <b>* * *</b>           * *          * *           * *   
2024-11-24 02:00:00  <b>* * *</b>           * *          * *           * *   
2024-11-24 03:00:00                         * *          * *           * *   
2024-11-24 04:00:00                         * *          * *           * *   
2024-11-24 05:00:00                                      * *           * *   
2024-11-24 06:00:00                                      * *           * *   
2024-11-24 07:00:00                                      * *           * *   
2024-11-24 08:00:00                                                          
2024-11-24 09:00:00                                                          
2024-11-24 10:00:00                                                          
2024-11-24 11:00:00                                                          
2024-11-24 12:00:00                                                          
2024-11-24 13:00:00                                                          

                    TIC 283940788 TIC 307119043 TIC 389836747 TIC 328181241  \
2024-11-24 01:00:00           * *           * *           * *                 
2024-11-24 02:00:00           * *  <b>* * *</b>           * *                 
2024-11-24 03:00:00           * *           * *           * *           * *   
2024-11-24 04:00:00           * *           * *  <b>* * *</b>           * *   
2024-11-24 05:00:00           * *           * *  <b>* * *</b>           * *   
2024-11-24 06:00:00           * *           * *  <b>* * *</b>           * *   
2024-11-24 07:00:00           * *           * *           * *           * *   
2024-11-24 08:00:00           * *           * *           * *           * *   
2024-11-24 09:00:00                                       * *           * *   
2024-11-24 10:00:00                                                           
2024-11-24 11:00:00                                                           
2024-11-24 12:00:00                                                           
2024-11-24 13:00:00                                                           

                    TIC 454140642 TIC 336882813  
2024-11-24 01:00:00                              
2024-11-24 02:00:00                              
2024-11-24 03:00:00                              
2024-11-24 04:00:00                              
2024-11-24 05:00:00           * *                
2024-11-24 06:00:00           * *           * *  
2024-11-24 07:00:00           * *           * *  
2024-11-24 08:00:00           * *  <b>* * *</b>  
2024-11-24 09:00:00           * *  <b>* * *</b>  
2024-11-24 10:00:00           * *  <b>* * *</b>  
2024-11-24 11:00:00                <b>* * *</b>  
2024-11-24 12:00:00                <b>* * *</b>  
2024-11-24 13:00:00                         * *

In [222]:
importlib.reload(pr)
foo = pl.categorized_priorities[0].style \
    .map(lambda val: "background-color: light-gray" if len(val)>3 else "background-color: black") \
    .format_index("{:%H:%M}")
foo.to_html()

'<style type="text/css">\n#T_79810_row0_col0, #T_79810_row0_col1, #T_79810_row1_col0, #T_79810_row1_col1, #T_79810_row1_col4, #T_79810_row2_col4, #T_79810_row2_col5, #T_79810_row3_col4, #T_79810_row3_col5, #T_79810_row4_col5, #T_79810_row4_col8, #T_79810_row5_col5, #T_79810_row5_col8, #T_79810_row6_col8, #T_79810_row7_col4, #T_79810_row7_col8, #T_79810_row7_col9, #T_79810_row8_col8, #T_79810_row8_col9, #T_79810_row9_col8 {\n  background-color: light-gray;\n}\n#T_79810_row0_col2, #T_79810_row0_col3, #T_79810_row0_col4, #T_79810_row0_col5, #T_79810_row0_col6, #T_79810_row0_col7, #T_79810_row0_col8, #T_79810_row0_col9, #T_79810_row1_col2, #T_79810_row1_col3, #T_79810_row1_col5, #T_79810_row1_col6, #T_79810_row1_col7, #T_79810_row1_col8, #T_79810_row1_col9, #T_79810_row2_col0, #T_79810_row2_col1, #T_79810_row2_col2, #T_79810_row2_col3, #T_79810_row2_col6, #T_79810_row2_col7, #T_79810_row2_col8, #T_79810_row2_col9, #T_79810_row3_col0, #T_79810_row3_col1, #T_79810_row3_col2, #T_79810_row3_co

In [138]:
tic = "TIC 470710327"
print(pl.target_list.target_list[pl.target_list.target_list["Target Name"] == tic]["Observable Max Alt"])
pl.tables[tic][0]

4225    60.595523
Name: Observable Max Alt, dtype: float64


AttributeError: 'PriorityList' object has no attribute 'tables'

In [225]:
target_list = pl.target_list.target_list
this_table_target_list = target_list[target_list["Target Name"].isin(ranking.columns)]
ra_order = list(this_table_target_list.sort_values("ra")["Target Name"])
extra_cols = ["RA", "Dec"]#, "Teff"]
for extra_col in extra_cols:
    ranking.loc[extra_col] = [target_list[target_list["Target Name"] == col][extra_col].values[0] for col in ra_order]
new_ordering = [*ranking.index[-len(extra_cols) :], *ranking.index[: -len(extra_cols)]]
final = ranking.reindex(new_ordering)
final.to_csv("final.csv")
final

NameError: name 'ranking' is not defined

In [186]:
tl.target_list[tl.target_list["Target Type"] == "RV Standard"].sort_values("ra")[["Target Name", "Teff", "RA", "Dec"]]

,Target Name,Teff,RA,Dec
5,HD 693,6750.0,00:11:15.86,-15:28:04.70
2,HD 3765,3800.0,00:40:49.27,+40:11:13.80
0,HD 18884,2000.0,03:02:16.77,+04:05:23.10
3,HD 23169,5200.0,03:43:53.14,+25:43:31.50
4,HD 65583,5800.0,08:00:32.13,+29:12:44.50
1,HD 92588,3600.0,10:41:24.19,-01:44:29.40
8,HD 102870,7350.0,11:50:41.72,+01:45:53.00
7,HD 112299,7100.0,12:55:28.29,+25:44:17.00
6,BD +28 3402,6900.0,19:35:00.36,+29:05:14.10
